# EDA v1 - RentSense

Анализ данных о предложениях аренды недвижимости в Москве.

Цель: понять структуру данных, выявить проблемы и подготовиться к обучению моделей.


In [ ]:
# Импорты библиотек
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import dotenv_values
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')

sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Библиотеки загружены")
print(f"NumPy: {np.__version__}, Pandas: {pd.__version__}, Matplotlib: {matplotlib.__version__}")


⚠️ Обнаружен NumPy 2.3.3 - версия 2.x несовместима
Устанавливаю NumPy <2...
✅ NumPy понижен до версии <2

🔴 ВАЖНО: Перезапустите ядро Jupyter сейчас!
   Kernel → Restart Kernel
   Затем запустите следующую ячейку


NumPy version: 1.26.4
✓ Pandas: 2.3.2
✓ Matplotlib: 3.9.2
✓ Seaborn: 0.13.2
✓ SQLAlchemy и другие библиотеки загружены

✅ Все библиотеки успешно загружены!


## 1. Подключение к БД и загрузка данных


In [ ]:
# Подключение к БД
from pathlib import Path
env_path = Path('../..') / '.env'
env = dotenv_values(env_path)

DBTYPE = env.get('DB_TYPE') or 'mysql+pymysql'
LOGIN = env.get('DB_LOGIN') or 'root'
PASS = env.get('DB_PASS') or 'rootpassword'
IP = env.get('DB_IP') or 'localhost'
PORT = env.get('DB_PORT') or '3307'
DBNAME = env.get('DB_NAME') or 'rentsense'

DATABASE_URL = f'{DBTYPE}://{LOGIN}:{PASS}@{IP}:{PORT}/{DBNAME}?charset=utf8mb4'
engine = create_engine(DATABASE_URL, pool_pre_ping=True)

print(f"Подключение к БД: {DBNAME}@{IP}:{PORT}")


In [ ]:
# Загрузка данных
query = """
SELECT 
    o.cian_id,
    o.price,
    o.category,
    o.views_count,
    o.photos_count,
    o.floor_number,
    o.floors_count,
    o.publication_at,
    o.created_at as offer_created_at,
    o.updated_at as offer_updated_at,
    
    a.county,
    a.district,
    a.street,
    a.house,
    a.metro,
    a.travel_type,
    a.travel_time,
    a.coordinates,
    
    ri.repair_type,
    ri.total_area,
    ri.living_area,
    ri.kitchen_area,
    ri.ceiling_height,
    ri.balconies,
    ri.loggias,
    ri.rooms_count,
    ri.separated_wc,
    ri.combined_wc,
    ri.windows_view,
    
    ro.build_year,
    ro.entrances,
    ro.material_type,
    ro.parking_type,
    ro.garbage_chute,
    ro.lifts_count,
    ro.passenger_lifts,
    ro.cargo_lifts,
    
    rd.realty_type,
    rd.project_type,
    rd.heat_type,
    rd.gas_type,
    rd.is_apartment,
    rd.is_penthouse,
    rd.is_mortgage_allowed,
    rd.is_premium,
    rd.is_emergency,
    
    od.deal_type,
    od.flat_type,
    od.payment_period,
    od.deposit,
    od.prepay_months,
    od.utilities_included,
    od.client_fee,
    od.agent_fee,
    od.description,
    
    d.name as developer_name,
    d.review_count as developer_review_count,
    d.total_rate as developer_rate,
    d.buildings_count as developer_buildings_count,
    d.foundation_year as developer_foundation_year,
    d.is_reliable as developer_is_reliable
FROM offers o
LEFT JOIN addresses a ON o.cian_id = a.cian_id
LEFT JOIN realty_inside ri ON o.cian_id = ri.cian_id
LEFT JOIN realty_outside ro ON o.cian_id = ro.cian_id
LEFT JOIN realty_details rd ON o.cian_id = rd.cian_id
LEFT JOIN offers_details od ON o.cian_id = od.cian_id
LEFT JOIN developers d ON o.cian_id = d.cian_id
"""

df = pd.read_sql(query, engine)
print(f"Загружено строк: {len(df)}")
df.head()


## 2. Общий обзор датасета


In [ ]:
print(f"Размер датасета: {df.shape}")
print(f"\nКолонки ({len(df.columns)}):")
print(df.columns.tolist())
print(f"\nТипы данных:")
print(df.dtypes)
print(f"\nОсновная информация:")
df.info()


In [ ]:
# Статистика по числовым признакам
numeric_cols = df.select_dtypes(include=[np.number]).columns
print("Описательная статистика числовых признаков:")
df[numeric_cols].describe()


In [ ]:
# Преобразование цены в float
df['price'] = pd.to_numeric(df['price'], errors='coerce')

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Гистограмма
axes[0, 0].hist(df['price'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение цен (гистограмма)')
axes[0, 0].set_xlabel('Цена (руб.)')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].grid(True, alpha=0.3)

# Boxplot
axes[0, 1].boxplot(df['price'].dropna(), vert=True)
axes[0, 1].set_title('Распределение цен (boxplot)')
axes[0, 1].set_ylabel('Цена (руб.)')
axes[0, 1].grid(True, alpha=0.3)

# Логарифмированная шкала
log_price = np.log1p(df['price'].dropna())
axes[1, 0].hist(log_price, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Распределение цен (логарифмированная шкала)')
axes[1, 0].set_xlabel('log(Цена + 1)')
axes[1, 0].set_ylabel('Частота')
axes[1, 0].grid(True, alpha=0.3)

# Q-Q plot для проверки нормальности
from scipy import stats
stats.probplot(df['price'].dropna(), dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q plot цены')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Статистика по цене:")
print(df['price'].describe())
print(f"\nМедиана: {df['price'].median():.2f}")
print(f"Среднее: {df['price'].mean():.2f}")
print(f"Мода: {df['price'].mode().values[0] if len(df['price'].mode()) > 0 else 'N/A'}")


### 3.2 Распределение числовых признаков


In [ ]:
# Основные числовые признаки для анализа
key_numeric = ['total_area', 'living_area', 'kitchen_area', 'floor_number', 
               'floors_count', 'build_year', 'rooms_count', 'ceiling_height']

# Фильтруем только те колонки, которые есть в датасете
key_numeric = [col for col in key_numeric if col in df.columns]

fig, axes = plt.subplots(len(key_numeric), 2, figsize=(15, 4 * len(key_numeric)))

for i, col in enumerate(key_numeric):
    if col in df.columns:
        data = pd.to_numeric(df[col], errors='coerce').dropna()
        
        if len(data) > 0:
            # Гистограмма
            axes[i, 0].hist(data, bins=30, edgecolor='black', alpha=0.7)
            axes[i, 0].set_title(f'Распределение {col}')
            axes[i, 0].set_xlabel(col)
            axes[i, 0].set_ylabel('Частота')
            axes[i, 0].grid(True, alpha=0.3)
            
            # Boxplot
            axes[i, 1].boxplot(data, vert=True)
            axes[i, 1].set_title(f'Boxplot {col}')
            axes[i, 1].set_ylabel(col)
            axes[i, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 3.3 Распределение категориальных признаков


In [ ]:
# Основные категориальные признаки
cat_cols = ['district', 'repair_type', 'material_type', 'realty_type', 
            'deal_type', 'flat_type', 'rooms_count']

# Фильтруем только те колонки, которые есть в датасете
cat_cols = [col for col in cat_cols if col in df.columns]

fig, axes = plt.subplots(len(cat_cols), 1, figsize=(15, 5 * len(cat_cols)))

for i, col in enumerate(cat_cols):
    if col in df.columns:
        value_counts = df[col].value_counts().head(15)  # Топ-15 значений
        
        axes[i].barh(range(len(value_counts)), value_counts.values)
        axes[i].set_yticks(range(len(value_counts)))
        axes[i].set_yticklabels(value_counts.index)
        axes[i].set_title(f'Распределение {col} (топ-15)')
        axes[i].set_xlabel('Количество')
        axes[i].grid(True, alpha=0.3, axis='x')
        
        # Поворачиваем длинные метки
        axes[i].invert_yaxis()

plt.tight_layout()
plt.show()

# Выводим статистику по категориальным признакам
for col in cat_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Уникальных значений: {df[col].nunique()}")
        print(f"  Пропусков: {df[col].isna().sum()} ({df[col].isna().sum() / len(df) * 100:.1f}%)")
        print(f"  Топ-5:")
        print(df[col].value_counts().head())


## 4. Анализ пропусков


In [ ]:
# Подсчет пропусков
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Пропусков': missing,
    'Процент': missing_percent
}).sort_values('Процент', ascending=False)

print("Колонки с пропусками:")
print(missing_df[missing_df['Пропусков'] > 0])

# Визуализация пропусков
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Heatmap пропусков (только колонки с пропусками)
cols_with_missing = missing_df[missing_df['Пропусков'] > 0].index[:30]  # Топ-30
if len(cols_with_missing) > 0:
    sns.heatmap(df[cols_with_missing].isnull(), yticklabels=False, 
                cbar=True, cmap='viridis', ax=axes[0])
    axes[0].set_title('Heatmap пропусков (топ-30 колонок)')
    
    # Bar plot процента пропусков
    top_missing = missing_df.head(20)
    axes[1].barh(range(len(top_missing)), top_missing['Процент'].values)
    axes[1].set_yticks(range(len(top_missing)))
    axes[1].set_yticklabels(top_missing.index, fontsize=8)
    axes[1].set_xlabel('Процент пропусков')
    axes[1].set_title('Топ-20 колонок по проценту пропусков')
    axes[1].grid(True, alpha=0.3, axis='x')
    axes[1].invert_yaxis()

plt.tight_layout()
plt.show()


### Стратегии заполнения пропусков

Предварительные рекомендации:
- **Числовые признаки**: медиана или среднее, в зависимости от распределения
- **Категориальные признаки**: мода или "unknown"
- **Координаты**: могут быть критичными для geo-фичей
- **build_year**: можно попробовать восстановить по району/материалу


## 5. Выбросы

### 5.1 IQR метод


In [ ]:
def detect_outliers_iqr(df, column):
    """Обнаружение выбросов методом IQR"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Анализ выбросов для ключевых числовых признаков
key_columns = ['price', 'total_area', 'living_area', 'floor_number', 'build_year']

outliers_summary = {}
for col in key_columns:
    if col in df.columns:
        df_col = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(df_col) > 0:
            outliers, lower, upper = detect_outliers_iqr(df, col)
            outliers_summary[col] = {
                'count': len(outliers),
                'percent': len(outliers) / len(df) * 100,
                'lower_bound': lower,
                'upper_bound': upper,
                'min_value': df[col].min(),
                'max_value': df[col].max()
            }

# Вывод результатов
print("Выбросы (IQR метод):")
for col, stats in outliers_summary.items():
    print(f"\n{col}:")
    print(f"  Выбросов: {stats['count']} ({stats['percent']:.1f}%)")
    print(f"  Границы: [{stats['lower_bound']:.2f}, {stats['upper_bound']:.2f}]")
    print(f"  Диапазон данных: [{stats['min_value']:.2f}, {stats['max_value']:.2f}]")

# Визуализация выбросов
fig, axes = plt.subplots(len(outliers_summary), 1, figsize=(12, 4 * len(outliers_summary)))

for i, (col, stats) in enumerate(outliers_summary.items()):
    data = pd.to_numeric(df[col], errors='coerce').dropna()
    axes[i].boxplot(data, vert=True)
    axes[i].axhline(y=stats['lower_bound'], color='r', linestyle='--', alpha=0.5, label='Lower bound')
    axes[i].axhline(y=stats['upper_bound'], color='r', linestyle='--', alpha=0.5, label='Upper bound')
    axes[i].set_title(f'Выбросы в {col} (IQR метод)')
    axes[i].set_ylabel(col)
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 5.2 Z-score метод


In [ ]:
def detect_outliers_zscore(df, column, threshold=3):
    """Обнаружение выбросов методом Z-score"""
    df_col = pd.to_numeric(df[column], errors='coerce')
    z_scores = np.abs((df_col - df_col.mean()) / df_col.std())
    outliers = df[z_scores > threshold]
    return outliers, z_scores

# Анализ выбросов Z-score
zscore_summary = {}
for col in key_columns:
    if col in df.columns:
        df_col = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(df_col) > 0 and df_col.std() > 0:
            outliers, z_scores = detect_outliers_zscore(df, col, threshold=3)
            zscore_summary[col] = {
                'count': len(outliers),
                'percent': len(outliers) / len(df) * 100,
                'max_z_score': z_scores.max() if len(z_scores) > 0 else 0
            }

print("Выбросы (Z-score метод, threshold=3):")
for col, stats in zscore_summary.items():
    print(f"{col}: {stats['count']} выбросов ({stats['percent']:.1f}%), max Z-score: {stats['max_z_score']:.2f}")


### 5.3 Анализ экстремальных значений цены

Проверим самые дорогие и дешевые предложения


In [ ]:
# Самые дорогие и дешевые предложения
print("Топ-10 самых дорогих:")
print(df.nlargest(10, 'price')[['cian_id', 'price', 'total_area', 'district', 'rooms_count', 'repair_type']].to_string())

print("\nТоп-10 самых дешевых:")
print(df.nsmallest(10, 'price')[['cian_id', 'price', 'total_area', 'district', 'rooms_count', 'repair_type']].to_string())

# Цена за квадратный метр
df['price_per_sqm'] = df['price'] / pd.to_numeric(df['total_area'], errors='coerce')

print("\nСтатистика цены за м²:")
print(df['price_per_sqm'].describe())

# Визуализация цены за м²
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].hist(df['price_per_sqm'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение цены за м²')
axes[0].set_xlabel('Цена за м² (руб.)')
axes[0].set_ylabel('Частота')
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(df['price_per_sqm'].dropna(), vert=True)
axes[1].set_title('Boxplot цены за м²')
axes[1].set_ylabel('Цена за м² (руб.)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 6. Утечка по времени

Проверяем, что train и test разделены по времени публикации (test позже train).


In [ ]:
# Преобразование publication_at из timestamp в datetime
df['publication_date'] = pd.to_datetime(df['publication_at'], unit='s', errors='coerce')

# Удаляем строки без даты публикации
df_with_date = df.dropna(subset=['publication_date']).copy()

print(f"Всего записей: {len(df)}")
print(f"Записей с датой публикации: {len(df_with_date)}")
print(f"\nДиапазон дат публикации:")
print(f"  От: {df_with_date['publication_date'].min()}")
print(f"  До: {df_with_date['publication_date'].max()}")

# Распределение по времени
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Гистограмма по датам
axes[0].hist(df_with_date['publication_date'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение публикаций по времени')
axes[0].set_xlabel('Дата публикации')
axes[0].set_ylabel('Количество публикаций')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Пример разделения train/test (последние 30 дней = test)
split_date = df_with_date['publication_date'].max() - pd.Timedelta(days=30)
train_mask = df_with_date['publication_date'] <= split_date
test_mask = df_with_date['publication_date'] > split_date

train_data = df_with_date[train_mask]
test_data = df_with_date[test_mask]

print(f"\nРазделение на train/test (split_date = {split_date.date()}):")
print(f"  Train: {len(train_data)} записей ({len(train_data)/len(df_with_date)*100:.1f}%)")
print(f"  Test: {len(test_data)} записей ({len(test_data)/len(df_with_date)*100:.1f}%)")

# Визуализация разделения
axes[1].scatter(train_data['publication_date'], train_data['price'], 
                alpha=0.3, label='Train', s=10)
axes[1].scatter(test_data['publication_date'], test_data['price'], 
                alpha=0.3, label='Test', s=10, color='red')
axes[1].axvline(x=split_date, color='green', linestyle='--', linewidth=2, label='Split date')
axes[1].set_title('Разделение train/test по времени')
axes[1].set_xlabel('Дата публикации')
axes[1].set_ylabel('Цена (руб.)')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Проверка, что test позже train
if len(train_data) > 0 and len(test_data) > 0:
    max_train_date = train_data['publication_date'].max()
    min_test_date = test_data['publication_date'].min()
    print(f"\nПроверка временного разделения:")
    print(f"  Max train date: {max_train_date}")
    print(f"  Min test date: {min_test_date}")
    print(f"  Test позже train: {min_test_date > max_train_date} ✓" if min_test_date > max_train_date else "  ⚠️ ПРОБЛЕМА: test содержит более старые данные!")


## 7. Корреляции

Анализ взаимосвязей между признаками


In [ ]:
# Выбираем числовые колонки для корреляционного анализа
numeric_for_corr = ['price', 'total_area', 'living_area', 'kitchen_area', 
                    'floor_number', 'floors_count', 'build_year', 'rooms_count',
                    'ceiling_height', 'views_count', 'photos_count']

numeric_for_corr = [col for col in numeric_for_corr if col in df.columns]

# Вычисляем корреляции
corr_matrix = df[numeric_for_corr].apply(pd.to_numeric, errors='coerce').corr()

# Визуализация
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица числовых признаков')
plt.tight_layout()
plt.show()

# Корреляции с ценой
if 'price' in corr_matrix.columns:
    price_corr = corr_matrix['price'].sort_values(ascending=False)
    print("\nКорреляции с ценой:")
    print(price_corr)


## 8. Выводы и рекомендации

### Основные находки:
1. **Размер датасета**: [заполнить после анализа]
2. **Пропуски**: [заполнить после анализа]
3. **Выбросы**: [заполнить после анализа]
4. **Временное разделение**: [заполнить после анализа]

### Рекомендации для подготовки данных:
1. **Очистка данных**:
   - Удалить/обработать выбросы в цене и площади
   - Заполнить пропуски стратегически
   
2. **Feature Engineering**:
   - Добавить цену за м²
   - Вычислить возраст дома
   - Добавить geo-фичи (расстояние до центра, метро)
   
3. **Разделение данных**:
   - Использовать временное разделение (последние N дней = test)
   - Убедиться, что нет утечки по времени

4. **Моделирование**:
   - Рассмотреть логарифмирование цены (если распределение логнормальное)
   - Использовать CatBoost/LightGBM для работы с пропусками
